### **8.7 - [Practica] Monitoring Airflow with TIG stack**

Let’s set up the TIG stack in order to monitor Airflow. First, check that you are under the folder airflow

materials/airflow-section-8 and open the file docker-compose-CeleryExecutor

TIG.yml. As usual, we have the different services to run Airflow with the CeleryExecutor. If

you scroll down,

you will reach the services of the TIG stack. Here,

there is Telegraf with a volume to bind the configuration file telegraf.conf from mnt

/telegraf to the service.

We will take a look at it in a minute. Just below, the InfluxDB service is initialized with default

ports set.

Finally, Grafana is defined as well on the port 3000.

Now let’s configure Telegraf. Open the file telegraf.conf in mnt/telegraf and look for

the section “outputs.influxdb”.

As you can see all the lines below

this comment correspond to the output plugins.

There are predefined and the only thing you have to do is uncomment what you want to turn on.

By default, the output influxdb is enabled but we have to configure it. First, uncomment the third line

“urls”, and replace the ip address to “influxdb”.

Remember that the values here correspond to the service name and the port of InfluxDB in the docker

compose file.

Next, uncomment the line with “database” in order to specify the target database where the metrics will

be set

which is “telegraf” by default. Uncomment the parameter skip_database_creation

and set it to “True”

since we are going to create the database by ourselves from the container of InfluxDB. Then, uncomment

the parameters timeout to define a timeout for HTTP messages.

Just below,

we uncomment username and password in order to define the user account that will be used by Telegraf

to send metrics into InfluxDB. Let’s keep the username “telegraf” and change the password to “telegraf

pass”.

Like that. Uncomment the user-agent as well. And that’s it,

the output plugin InfluxDB is configured and Telegraf is ready to send data to it. So, the output

is set but the input where Telegraf will receive the metrics coming from Airflow is still missing.

Let's fix this.

Look for the section “inputs.statsd”

and let’s uncomment all the lines of the section.

Like that.

I’m not going to explain each parameter since there is already a description above each one but basically

Telegraf will start a StatsD daemon listening on the port 8125 in UDP for metrics coming from Airflow.

Save the file and let’s configure Airflow so that it will send the metrics to Telegraf. Open the file

airflow.cfg and look for the section “statsd”. Here

there are 4 parameters that you have to define.

First, change the value of statsd_on from False to True in order to activate it.

Then, replace localhost here, by telegraf which is the name of the container running

Telegraf.

Finally, keep the two last parameters with their default values.

Perfect, now Airflow is configured to send its metrics into the input plugin statsd of Telegraf. Last thing,

look for the parameter remote_logging

and set it to False

if it’s not already done. Since we are not going to store the log files in a remote storage, there is

no need to keep it active.

Save the file.

All right.

Time to start everything.

Open your terminal, and check that you are under the folder airflow-materials/airflow-section

-8 and type “docker-compose -

f

docker-compose-CeleryExecutorTIG.yml up -d”.

Enter.

Wait a second or two.

Type “docker ps” to check that everything is running.

Perfect.

Now we have to create the database telegraf as well as the user telegraf in InfluxDB which will

be used by telegraf for sending the metrics. Type “docker exec

-it”

copy and past the container id of InfluxDB,

and type “influx”.

Enter.

And here we are connected to InfluxDB through the InfluxDB shell as shown here.

Type “create database telegraf”

Enter. Ok then create the user by executing “create user telegraf

with password

‘telegrafpass’”

Enter. And the user telegraf is created. Exit the docker container by hitting control-D, and open your

web browser. Type “localhost:3000”.

Enter. Type “admin” for both login and password.

Then “Log In”. We skip the step, and we land on the beautiful Grafana dashboard. From there, there are two

steps we need to follow as shown here. Let’s add a data source corresponding to the InfluxDB instance

we set up where the metrics of Airflow are stored. Click on “add data source”. Then select “InfluxDB”. Ok.

The URL is “http://influxdb:8086”.

Next.

under the section InfluxDB details, the database is “telegraf”,

the user is “telegraf” as well.

And the password is defined to “telegrafpass”.

I advise to add the “pass” here and just copy and paste the value so that you are sure the password

is correct.

Okay,

check that you have the settings as mine and click on “save and test”.

If everything works fine you should obtain the same message here telling you that the data source is

working.

Go back to the Grafana homepage by clicking here then “new dashboard” and “add query”.

Here, you have to define the query that will fetch the metrics you want from InfluxDB.

For example, if we click on “select measurement”. All the measurements prefixed by “airflow_” correspond

to the different metrics available of Airflow. Before moving forward,

open a new tab, and go to “https://

airflow.apache.org

/docs/stable/metrics.html”.

If you scroll down, you obtain the list of metrics as well as their descriptions all divided by types as

shown here.

Let’s say we want to know the number of DAGs loaded by Airflow. Under “Gauges” the metric dagbag

_size gives us this information. Since a dabbag corresponds to the loaded DAGs of the scheduler,

we actually need to active the different DAGs. Open a new tab, and type localhost:8080.

Enter. Then here,

turn on the toggle of both DAGs.

Perfect.

So back to Grafana,

select “airflow_dagbag_size”. Like that. Click on mean

here, “remove”. Click on the “plus”, “Selector” and “last”, so that we obtain last number of loaded DAGs

by the scheduler.

Remove the group by since the last number is fetched from the entire table. Then from left click on

“Visualization”, select the Gauge. Under display, click on Mean, Select Last, and we obtain a beautiful gauge

with the number 2 corresponding to dags logger_dag and data_dag imported

by airflow.

Here, you can customize the visualization if you want by setting the display,

the thresholds and so on. Next, click on “General” and set the title “Number of imported DAGs” so that

we know the meaning of the chart.

Okay.

Finally, at the top of the page, click on “Save dashboard”,

give the name “Airflow”,

click on “Save”. And you just have create your Dashboard for monitoring your Airflow instance with Grafana,

InfluxDB and Telegraf.

Well done! All right,

keep everything running and see you in the next video where I will show you how to be alerted if anything

goes wrong.
